Resources

---



https://huggingface.co/transformers/v4.3.3/_modules/transformers/training_args.html#:~:text=[docs]%20@dataclass%20class%20TrainingArguments:%20%22%22%22%20TrainingArguments%20is%20the%20subset%20of

https://github.com/NielsRogge/Transformers-Tutorials/blob/master/Table%20Transformer/Using_Table_Transformer_for_table_detection_and_table_structure_recognition.ipynb

https://pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html?highlight=cuda#cuda-tensors

https://github.com/microsoft/table-transformer/blob/main/docs/INFERENCE.md

https://camelot-py.readthedocs.io/en/master/

https://github.com/tesseract-ocr/tesseract

https://www.kaggle.com/datasets/sreesankar711/pubtables-subset-100k

https://github.com/microsoft/table-transformer

In [16]:
# Install packages
#!pip install camelot-py[cv] ghostscript PyPDF2==1.26.0 pdf2image transformers matplotlib numpy opencv-python pandas torch tensorflow table-transformer transformers datasets
!pip install camelot-py[cv] ghostscript PyPDF2==1.26.0 pdf2image transformers torch timm
!apt-get install -y ghostscript poppler-utils
#!apt-get install -y ghostscript
!pip install --upgrade camelot-py[cv]
#!pip install pytesseract
#!apt-get install -y tesseract-ocr
#!apt-get install -y poppler-utils
#!pip install --upgrade pillow datasets
#!pip install timm
#!kaggle datasets download -d sreesankar711/pubtables-subset-100k
#!unzip pubtables-subset-100k.zip -d /content/pubtables



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ghostscript is already the newest version (9.55.0~dfsg1-0ubuntu5.9).
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
  Using cached camelot_py-0.11.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached pypdf-5.0.1-py3-none-any.whl.metadata (7.4 kB)
INFO: pip is looking at multiple versions of camelot-py[cv] to determine which version is compatible with other requirements. This could take a while.
  Using cached camelot_py-0.10.1-py3-none-any.whl.metadata (8.3 kB)
  Using cached camelot_py-0.10.0-py3-none-any.whl.metadata (8.3 kB)


In [31]:
# Import libraries
import os
import json
import camelot
import ghostscript
import PyPDF2 as pypdf
from pdf2image import convert_from_path
from PIL import Image
from google.colab import files
import torch
import cv2
import numpy as np
import pandas as pd




In [32]:
# Load pre-trained Table Transformer model and image processor
try:
    model = TableTransformerForObjectDetection.from_pretrained('microsoft/table-transformer-detection')
    image_processor = AutoImageProcessor.from_pretrained("microsoft/table-transformer-detection")
except Exception as e:
    print(f"Error loading the model: {e}")
    print("Falling back to Camelot-only extraction method.")
    model = None
    image_processor = None

Error loading the model: 
TableTransformerConvEncoder requires the timm library but it was not found in your environment. You can install it with pip:
`pip install timm`. Please note that you may need to restart your runtime after installation.

Falling back to Camelot-only extraction method.


In [21]:
# Upload PDF file
upload = files.upload()
pdf_file = list(upload.keys())[0]

Saving 1_simple_table.pdf to 1_simple_table (1).pdf


In [23]:
def detect_tables(image):
    if model is None or image_processor is None:
        return []  # Return empty list if model is not available

    # Prepare image for the model
    inputs = image_processor(images=image, return_tensors="pt")

    # Perform table detection
    with torch.no_grad():
        outputs = model(**inputs)

    # Post-process outputs to get bounding boxes
    target_sizes = torch.tensor([image.size[::-1]])
    results = image_processor.post_process_object_detection(outputs, threshold=0.7, target_sizes=target_sizes)[0]

    return results['boxes'].tolist()

In [27]:
def extract_pdf_table(pdf_file):
    # Convert PDF to images
    images = convert_from_path(pdf_file, poppler_path="/usr/bin", fmt='png')

    all_tables = []

    for i, image in enumerate(images):
        # Detect tables in the image if the model is available
        bboxes = detect_tables(image) if model is not None else []

        if not bboxes:
            # If no tables detected or model not available, use Camelot on the whole page
            tables = camelot.read_pdf(pdf_file, pages=str(i+1), flavor='stream')
            all_tables.extend([table.df for table in tables])
        else:
            # Convert image to OpenCV format
            cv_image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

            for j, bbox in enumerate(bboxes):
                # Crop the image to the detected table region
                x1, y1, x2, y2 = [int(coord) for coord in bbox]
                cropped_image = cv_image[y1:y2, x1:x2]

                # Save the cropped image temporarily
                temp_image_path = f'temp_table_{i}_{j}.png'
                cv2.imwrite(temp_image_path, cropped_image)

                # Extract table from the cropped image using Camelot
                try:
                    tables = camelot.read_pdf(temp_image_path, flavor='stream')
                    if tables:
                        all_tables.extend([table.df for table in tables])
                except Exception as e:
                    print(f"Error extracting table from image {i}, bbox {j}: {e}")

                # Remove temporary file
                os.remove(temp_image_path)

    return all_tables

In [35]:
# Convert table to JSON
def json_table(table):
  # Check if table has DataFrame 'df' attribute
  if hasattr(table, 'df'):
    header = table.df.values.tolist() # Get df header
    rows = table.df.values.tolist() # Get df rows
    # Return list of dictionaries
    return [dict(zip(header, row)) for row in rows]
  else:
    # If table is a list or tuple
    if table and isinstance(table, (list, tuple)) and len(table) > 1:
      header = table[0] # First element is header
      rows = table[1:] # Remaining elements are rows
      # Return list of dictionaries
      return [dict(zip(header, row)) for row in rows]
    else:
      print("Invalid table format for JSON conversion.")
      return []

PdfReadError: Could not find xref table at specified location

In [29]:
# Extract tables from PDF file
tables = extract_pdf_table(pdf_file)

# Convert extracted tables to JSON
json_tables = [json_table(table) for table in tables]

# Save JSON to file
with open('tables.json', 'w') as f:
    json.dump(json_tables, f, indent=2)

print(f"Extracted {len(json_tables)} tables and saved as JSON.")

# Display first table as an example
if json_tables:
    print("\nFirst extracted table:")
    print(json.dumps(json_tables[0], indent=2))
else:
    print("No tables were extracted.")

NameError: name 'check_pdf' is not defined

In [30]:
# Install packages
!pip install camelot-py[cv] ghostscript PyPDF2==1.26.0 pdf2image transformers torch timm
!apt-get install -y ghostscript poppler-utils
!pip install --upgrade camelot-py[cv]

# Import libraries
import os
import json
import camelot
import ghostscript
import PyPDF2 as pypdf
from pdf2image import convert_from_path
from PIL import Image
from google.colab import files
import torch
import cv2
import numpy as np
import pandas as pd

# Try importing the required modules, with error handling
try:
    from transformers import TableTransformerForObjectDetection, AutoImageProcessor
except ImportError as e:
    print(f"Error importing from transformers: {e}")
    print("Please make sure you have the latest version of transformers installed.")
    raise

# Upload PDF file
upload = files.upload()
pdf_file = list(upload.keys())[0]

# Load pre-trained Table Transformer model and image processor
try:
    model = TableTransformerForObjectDetection.from_pretrained('microsoft/table-transformer-detection')
    image_processor = AutoImageProcessor.from_pretrained("microsoft/table-transformer-detection")
except Exception as e:
    print(f"Error loading the model: {e}")
    print("Falling back to Camelot-only extraction method.")
    model = None
    image_processor = None

def detect_tables(image):
    if model is None or image_processor is None:
        return []  # Return empty list if model is not available

    # Prepare image for the model
    inputs = image_processor(images=image, return_tensors="pt")

    # Perform table detection
    with torch.no_grad():
        outputs = model(**inputs)

    # Post-process outputs to get bounding boxes
    target_sizes = torch.tensor([image.size[::-1]])
    results = image_processor.post_process_object_detection(outputs, threshold=0.7, target_sizes=target_sizes)[0]

    return results['boxes'].tolist()

def extract_pdf_table(pdf_file):
    # Convert PDF to images
    images = convert_from_path(pdf_file, poppler_path="/usr/bin", fmt='png')

    all_tables = []

    for i, image in enumerate(images):
        # Detect tables in the image if the model is available
        bboxes = detect_tables(image) if model is not None else []

        if not bboxes:
            # If no tables detected or model not available, use Camelot on the whole page
            tables = camelot.read_pdf(pdf_file, pages=str(i+1), flavor='stream')
            all_tables.extend([table.df for table in tables])
        else:
            # Convert image to OpenCV format
            cv_image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

            for j, bbox in enumerate(bboxes):
                # Crop the image to the detected table region
                x1, y1, x2, y2 = [int(coord) for coord in bbox]
                cropped_image = cv_image[y1:y2, x1:x2]

                # Save the cropped image temporarily
                temp_image_path = f'temp_table_{i}_{j}.png'
                cv2.imwrite(temp_image_path, cropped_image)

                # Extract table from the cropped image using Camelot
                try:
                    tables = camelot.read_pdf(temp_image_path, flavor='stream')
                    if tables:
                        all_tables.extend([table.df for table in tables])
                except Exception as e:
                    print(f"Error extracting table from image {i}, bbox {j}: {e}")

                # Remove temporary file
                os.remove(temp_image_path)

    return all_tables

# Convert table to JSON
def json_table(table):
    if isinstance(table, pd.DataFrame):
        return table.to_dict(orient='records')
    else:
        print("Invalid table format for JSON conversion.")
        return []

# Extract tables from PDF file
tables = extract_pdf_table(pdf_file)

# Convert extracted tables to JSON
json_tables = [json_table(table) for table in tables]

# Save JSON to file
with open('tables.json', 'w') as f:
    json.dump(json_tables, f, indent=2)

print(f"Extracted {len(json_tables)} tables and saved as JSON.")

# Display first table as an example
if json_tables:
    print("\nFirst extracted table:")
    print(json.dumps(json_tables[0], indent=2))
else:
    print("No tables were extracted.")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ghostscript is already the newest version (9.55.0~dfsg1-0ubuntu5.9).
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
  Using cached camelot_py-0.11.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached pypdf-5.0.1-py3-none-any.whl.metadata (7.4 kB)
INFO: pip is looking at multiple versions of camelot-py[cv] to determine which version is compatible with other requirements. This could take a while.
  Using cached camelot_py-0.10.1-py3-none-any.whl.metadata (8.3 kB)
  Using cached camelot_py-0.10.0-py3-none-any.whl.metadata (8.3 kB)


Saving 1_simple_table.pdf to 1_simple_table (2).pdf
Error loading the model: 
TableTransformerConvEncoder requires the timm library but it was not found in your environment. You can install it with pip:
`pip install timm`. Please note that you may need to restart your runtime after installation.

Falling back to Camelot-only extraction method.


PdfReadError: Could not find xref table at specified location